In [ ]:
import numpy as np

# Para tratamiento y e/s de datos
import pandas as pd
# from pandas_profiling import ProfileReport

# Gráficos de datos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Conversión .ipynb ---> .py :
# Tener instalado: ipython, nbconvert
# En la terminal de anaconda: jupyter nbconvert [archivo].ipynb --to python

# Forecasting Demanda de Energía (Holt-Winters)

### Leemos los datos que tenemos

In [ ]:
# Importo el archivos de datos de consumo de energia en la zona este de EE.UU.
df = pd.read_csv(r'\kaggle_forecasting_EP\PJME_hourly.csv')

In [ ]:
df.head()

<b>Target_values: "PJME_MW"</b>

In [ ]:
#Convierto el index en DateTimeIndex
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis = 0, ascending = True, inplace = True)
df.reset_index(inplace = True, drop = True)

# La variable objetivo (y) se renombra a: demand_in_MW
df.rename(columns={'PJME_MW':'demand_in_MW'}, inplace = True)

df.head()

In [ ]:
df.shape

## Limpieza de datos

### Eliminación de datos duplicados

In [ ]:
# De datos duplicados, solo se mantiene la medición más reciente. 
df.drop_duplicates(subset = 'Datetime', keep = 'last', inplace = True)
df.shape

### Tratando los espacios vacios

<blockquote> 
El tratamiento se hace para tener
un grupo de datos continuos. 
</blockquote>

In [ ]:
df_2 = df.set_index('Datetime')
# print(f'df.index.freq is set to: {df.index.freq}')

In [ ]:
print(f'df_2.index.freq is set to: {df_2.index.freq}')

<i>
Tener un dataset con frecuencia en "None" indica 
que existen datos que perdidos (missing). <br>
Para verificar lo dicho, podemos comparar con un rango de datos
custom e ininterrumpido
</i>

In [ ]:
# Custom range
data_range = pd.date_range(start = min(df_2.index),
                          end = max(df_2.index),
                          freq = 'H') 
#freq = 'H' indica frecuencia por hora.
#Explicación: genero un dataframe con una frecuencia horaria desde el valor minimo del index (datetime)
#del dataframe original, y con el valor máximo del index. Con esto lo que obtengo es TODO EL CALENDARIO
#sin datos perdidos. 
#Al hacer mas adelante la diferencia entre ambos dataframe, voy a obtener los "días perdidos" del dataframe original. 
# https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
data_range

In [ ]:
print(f'La diferencia de longitud entre el rango customizado de datos y nuestro dataset es {(len(data_range)-len(df_2))}:')
print(data_range.difference(df_2.index))
#la diferencia entre ambos df indica la cantidad de valores perdidos en el df_original

<i>
Re-indexamos los valores al dataset original
</i>

In [ ]:
# El siguiente comando adjunta los datos "datetime" perdidos (missing) al dataset original
# pero va a generar valores NaN para la variable Target (Demand_in_MW)
df_3 = df_2.reindex(data_range)

# Llenamos estos valores blancos con valores que se encuentran en una curva lineal entre 
# puntos de datos existentes
df_3['demand_in_MW'].interpolate(method='linear', inplace=True)

# Con la interpolación se tiene un datetime (set de hora y dias) continuo
print(f'La df.index.freq ahora es: {df_3.index.freq}, indicando que ya no tenemos valores perdidos')

## Extraemos características de la variable Tiempo
<i>
Podemos dividir la columna de Datetime en sus diferentes componentes. <br>
Esto nos permite encontrar patrones para diferentes grupos.
</i>

In [ ]:
df_3['dow'] = df_3.index.day_of_week
df_3['doy'] = df_3.index.day_of_year
df_3['year'] = df_3.index.year
df_3['month'] = df_3.index.month
df_3['quarter'] = df_3.index.quarter
df_3['hour'] = df_3.index.hour
df_3['weekday'] = df_3.index.day_name()
df_3['woy'] = df_3.index.isocalendar().week #week of year
df_3['dom'] = df_3.index.day # Day of Month
df_3['date'] = df_3.index.date 

# número de estación del año
df_3['season'] = df_3['month'].apply(lambda month_number: (month_number%12 + 3)//3) 
# el operador aritmético // solo devuelve a parte entera de la división.

In [ ]:
df_3.index.year

## EDA (Exploratory data analysis)

In [ ]:
# ProfileReport(df_3)

### Gráficos
<br>
<b>Graficando el consumo de energía a lo largo del tiempo</b>

In [ ]:
#Plotyle no permite acceso directo a los index del df. 
df_3['date_and_time'] = df_3.index 

#Plotting
fig = px.line(df_3, x=['date_and_time'], y='demand_in_MW', title=f'Demanda MW por tiempo [{min(df_3.year)} - {max(df_3.year)}]')
fig.update_traces(line=dict(width=0.05))
fig.update_layout(xaxis_title='Date & Time', yaxis_title='Demanda Energía [MW]')
fig.show()

Estudiando la gráfica se observa un comportamiento con patron en temporadas (estación del año). 

### Patrones de fecha y hora. 

Podemos usar nuestras funciones de fecha y hora extraídas previamente <br>
para ver si surgen patrones recurrentes de los datos agregados. <br>
Tomemos, por ejemplo, la demanda de energía a lo largo del día para cada día de la semana:

In [ ]:
df_3.columns

In [ ]:
# Dataframe definido para reflejar el consumo por hora en la semana, usando la mediana de energia. 
patron_1 = df_3.groupby(['hour', 'weekday'], as_index=False).agg({'demand_in_MW':'median'})
patron_1

In [ ]:
fig = px.line(patron_1, 
              x = 'hour',
              y = 'demand_in_MW', 
              color='weekday', 
              title='Mediana de consumo de energia por hs por día de semana ')

fig.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[MW]')

fig.show()

El consumo de energía es más elevado los días de semana en comparación con los findes. 

In [ ]:
# Dataframe definido para graficar el consumo horario por temporada del año. Mediana de la energía. 
patron_2 = df_3.groupby(['hour', 'season'], as_index=False).agg({'demand_in_MW':'median'})

In [ ]:
fig_2 = px.line(patron_2, 
                x = 'hour',
                y = 'demand_in_MW', 
                color='season', 
                title='Mediana de consumo de energia por hs por estación')

fig_2.update_layout(xaxis_title='Hour', yaxis_title='Energy Demand[MW]')

fig_2.show()

Durante el verano le dan duro al aire acondicionado. 

## Descompoción de la serie de tiempo

Los puntos que representan datos a lo largo de una serie de tiempo pueden ser interesantes <br>
en cuanto sus patrones se complementes con tendencias de subida/bajada y/o estacionalidad. <br>
Según la info adquirida en el EDA esto parece ser así. <br>
<br>
Debido a que la variación por estaciones del año parece constante a lo largo del tiempo, <br>
usaremos el <b> modelo de sumas por descomposición </b> a diferencia del <br>
<b>modelo multiplicativo</b> que es útil para casos donde la variación incrementa en el tiempo.
<br>
Extracto de texto sacado de: __[PennState](https://online.stat.psu.edu/stat510/lesson/5/5.1)__
<br>
<blockquote>
The following two structures are considered for basic decomposition models:
<ol>
<li>Additive:  = Trend + Seasonal + Random</li>
<li>Multiplicative:  = Trend * Seasonal * Random</li>
</ol>
</blockquote>

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# seasonal_decompose necesita un dataframe con un index en formato datatime
series = df_3[['demand_in_MW']] #devuelve un dataframe. Si fuese df_3['demand_in_MW'] devuelve una serie
#Añadido: la season (patrón de movimiento) es anual, con un periodo(fre) igual a la cantidad de hs por año.
frequency = 24*365

# Descomposición de la serie de tiempo, con una freq (hs)= 24 hs por 365 días. 
# El modelo aditivo parace ser el mças acertado dado que no poseo una tendencia de 
# incremento/decremento en la gráfica. 
decomposed = seasonal_decompose(series, model='additive', period=frequency)
#obtengo un objeto con los siguientes atributos: temporada, tendencia y residuo:
# ------------- atributos = [[decomposed.seasonal], [decomposed.trend], [decomposed.resid]] -------------

In [ ]:
df_4 = df_3.copy()

In [ ]:
# df_trend = pd.DataFrame(decomposed.trend)
# df_seasonal = pd.DataFrame(decomposed.seasonal)
# df_resid = pd.DataFrame(decomposed.resid)
# df_4 = pd.concat([df_3, df_trend, df_seasonal, df_resid], axis=0)
df_4['Trend'] = decomposed.trend
df_4['Seasonal'] = decomposed.seasonal
df_4['Residuos'] = decomposed.resid

In [ ]:
df_4.columns

In [ ]:
fig_t = px.line(df_4,
                y = 'Trend',
                title='Trend')

# adjust line width
fig_t.update_traces(line=dict(width=2))
        
# change layout of axes and the figure's margins 
# to emulate tight_layout
fig_t.update_layout(
                xaxis=dict(showticklabels=False, linewidth=1),
                yaxis=dict(title=''),
                margin=go.layout.Margin(l=40, r=40, b=0, t=40, pad=0)
)

fig_t.show()

In [ ]:
fig_s = px.line(df_4,
                y = 'Seasonal',
                title='Seasonality')

# adjust line width
fig_s.update_traces(line=dict(width=0.025))
        
# change layout of axes and the figure's margins 
# to emulate tight_layout
fig_s.update_layout(
                xaxis=dict(showticklabels=False, linewidth=1),
                yaxis=dict(title=''),
                margin=go.layout.Margin(l=40, r=40, b=0, t=40, pad=0)
)

fig_s.show()

In [ ]:
fig_r = px.line(df_4,
                y = 'Residuos',
                title='Residuos')

# adjust line width
fig_r.update_traces(line=dict(width=0.05))
        
# change layout of axes and the figure's margins 
# to emulate tight_layout
fig_r.update_layout(
                xaxis=dict(showticklabels=False, linewidth=1),
                yaxis=dict(title=''),
                margin=go.layout.Margin(l=40, r=40, b=0, t=40, pad=0)
)

fig_r.show()

#### Función en desuso

<i>
El siguiente código fue escrito por quienes desarrollaron el proyecto, <br>
sin embargo contenía errores que me llevaron a resolver los gráficos de otra manera. <br>
No obstante me tome el tiempo de corregir y actualizar el bloque para que también corrieses. 
</i>

<br>

<blockquote>
def plot_decompositions(decompositions, titles, line_widths):
    for d, t, lw in zip(decompositions, titles, line_widths):
      
        
        fig = px.line(df_4, 
                      y=d,
                      title = t,
                      height=300)
        
        
        fig.update_traces(line=dict(width=lw))
        
        
        fig.update_layout(
            xaxis=dict(
                    showticklabels=False,
                    linewidth=1
            ),
            yaxis=dict(title=''),
            margin=go.layout.Margin(
                l=40, r=40, b=0, t=40, pad=0
            )
        )
              
        fig.show()


plot_decompositions([df_3.Trend, df_3.Seasonal, df_3.Residuos], ['Trend','Seasonality','Residuos'],[2, 0.025, 0.05])
</blockquote>

# Modelos de estimación (Forecasting)

Se tendrán en cuenta los siguientes modelos:
    <ol>
    <li> Triple Exponential Smoothing: Holt-Winters </li>
    <li> Explicit Multi-Seasonality: Prophet </li>
    </ol>

## Train/Test
<b>Objetivo</b>: Estimación precisa de la demanda de los proximos 12 meses. <br>
Para lograr el objetivo, restringimos los datos de "entrenamiento" a algunos años antes de la fecha max.<br>
Esto lo hacemos para evitar tendencias que jodan al modelo (por lo analizado, no parece ser el caso)

In [ ]:
# Opción alternativa
# Agrego esta libreria como opción alternativa al proyecto 
# para configurar los datos de train/test 
# from sklearn.model_selection import train_test_split

In [ ]:
print(f'El primer punto de medicion fecha/hs es: {min(df_3.index)}')
print(f'El último punto de medicion fecha/hs es: {max(df_3.index)}')

In [ ]:
# Dataframe de recort
CUTOFF_DATE = pd.to_datetime('2017-08-01')
TIME_DELTA = pd.DateOffset(years=8)

# Separo df p/ test y df p/ train
train = df_3.loc[(df_3.index < CUTOFF_DATE) & (df_3.index >= CUTOFF_DATE - TIME_DELTA)].copy()
test = df_3.loc[df_3.index >= CUTOFF_DATE].copy()

In [ ]:
#Se permite recortar varias fechas porque:
#1- El comportamiento es constante en el tiempo.
#2- Alivia la carga de procesamiento en la PC.
print(f'Training shape: {train.shape}\n Testing shape: {test.shape}\n')
print(f'Las fechas de entrenamiento son: {min(train.index)} & {max(train.index)}')
print(f'Las fechas de test son: {min(test.index)} & {max(test.index)}')

# Holt-Winter: Suavizado exponencial triple -> valores, tendencias y temporada. 

Teoría: __[Suavización exponencial](https://economipedia.com/definiciones/suavizacion-exponencial.html)__ <br>
Teoría: __[Holt Winter](http://cienciauanl.uanl.mx/?p=7948#:~:text=El%20m%C3%A9todo%20Holt%2DWinters%20es,de%20pron%C3%B3sticos%20a%20corto%20plazo.)__ <br>
Teoría: __[+ Holt Winter (excelente)](https://orangematter.solarwinds.com/2019/12/15/holt-winters-forecasting-simplified/#:~:text=Holt%2DWinters%20is%20a%20model,cyclical%20repeating%20pattern%20(seasonality).)__
<br>
<i>Este metodo es una ampliación perfeccionada del enfoque de la <b>suavización exponencial</b>:</i>
<blockquote>
El método de suavización exponencial utiliza los promedios históricos de una <b>variable</b> en un 
período para intentar predecir su comportamiento futuro.<br>
Para estimar lo que va a suceder con la variable necesita suavizar su <b>Serie temporal</b>, es decir, al conjunto de datos
que describen a la variable ordenados cronológicamente (vital). <br>
El objetivo de la suavización es reducir las fluctuaciones y conseguir observar una tendencia que a veces no está
clara a simple vista. <br>
<b>Método de suavización exponencial:</b><br>
Ejemplo en la ecuación: "predicción de demanda" -> <br>Do: demanda real; Po: pronóstico; alfa: factor de suavización<br>
<b><i>P1 = Po + alfa(Do-Po)</i></b><br>
Métodos que usan esta tecnica: Box-Jenkins y Holt-Winter<br>
</blockquote>

In [ ]:
import statsmodels.api as sm
# from statsmodels.tools.sm_exceptions import ConvergenceWarning
# import warnings
# warnings.simplefilter('ignore', ConvergenceWarning)
# El método de suavizadoo exponencial triple solo considera patrones dentro de la variable objetivo (demanda)

# Defino los argumentos que va a tomar el modelo Holt-Winter:
# endog: La serie de tiempo a modelar (exp_smooth_train)
# la serie para testear el resultado: exp_smooth_test

exp_smooth_train, exp_smooth_test = train['demand_in_MW'], test['demand_in_MW']

# Predicción
holt_winter = sm.tsa.ExponentialSmoothing(exp_smooth_train,
                                          seasonal_periods=24*365,
                                          seasonal = 'add').fit() 


The fit() function will return an instance of the HoltWintersResults class that contains the learned coefficients. 
The forecast() or the predict() function on the result object can be called to make a forecast.

In [ ]:
print(len(exp_smooth_test))

In [ ]:
periodo_prueba = len(exp_smooth_test)*4 #estiro el periodo de test a 4 años, donde los ultimos 3 no tienen datos. 
y_hat_holt_winter = holt_winter.forecast(periodo_prueba)
# y_hat_holt_winter[-8809:]

## Guardo los datos obtenidos en un archivo .csv

In [ ]:
df_y = pd.DataFrame(y_hat_holt_winter)
df_y.rename(columns={0:'Demanda [MW]'}, inplace = True)
df_y.index.names = ['DateTime']
df_y

In [ ]:
df_y.to_csv('forecasting_holtwinter.csv', encoding='utf-8')

In [ ]:
# Gráfica
fig_hw = go.Figure()
fig_hw.add_trace(go.Scatter(x=exp_smooth_train.index, y=exp_smooth_train,
                         mode='lines',
                         name='Train'))
fig_hw.add_trace(go.Scatter(x=exp_smooth_test.index, y=exp_smooth_test,
                         mode='lines',
                         name='Test - Ground Truth'))
fig_hw.add_trace(go.Scatter(x=y_hat_holt_winter.index, y=y_hat_holt_winter,
                         mode='lines', 
                         name='Test - Prediction'))

# adjust layout
fig_hw.update_traces(line=dict(width=0.5))
fig_hw.update_layout(title='Holt-Winter Forecast of Hourly Energy Demand',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')

## ConvergenceWarning <br>
Link: __[Wiki](https://es.wikipedia.org/wiki/Convergencia)__ <br>
<blockquote>
En matemática, la convergencia es una propiedad de las sucesiones que tienden a un límite. <br>
En estadística convergencia en probabilidad es la aparición de patrones en la probabilidad de una variable aleatoria según aumenta la muestra.<br>
</blockquote>
Ignore la advertencia (warning) sobre el error de convergencia ---> investigar sobre la cantidad de datos
que necesito para tener una aproximación mas certera

En comparación con un dataset de test/train con muchos mas valores para "train", la gráfica anterior parece ser más precisa. <br>
<b>Tengo que entender porqué el error de convergencia</b>

## Análisis de lo obtenido con Holt-Winters

Reconoce el patrón de los datos que tenemos, pero durante la época de invierno (enero/2018) y los ultimos 
meses de verano (sep/2017) se queda algo corto

### Evaluación de métricas

Cuantificamos el rendimiento del método (se hace como paso natural del proceso)

Un análisis de error basado en tres medidas estadísticas se emplea para estimar el rendimiento y la confiabilidad del modelo:<br>
<ol>
<li>error de porcentaje absoluto medio (MAPE),</li>
<li>desviación absoluta media (MAD),</li>
<li>y error cuadrático medio ó desviación cuadrática media (MSD)</li>
</ol>

In [ ]:
def mape(y_true, y_pred):
    """Error de porcentaje absoluto medio"""
    
    # conversión a vectores numpy
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Porcentaje de error
    pe = (y_true - y_pred) / y_true
    
    # valor absolutos
    ape = np.abs(pe)
    
    # Cuantificación del rendimiento en un solo nº
    mape = np.mean(ape)
    
    return f'{mape*100:.2f}%'

In [ ]:
print(f'Error del: {mape(exp_smooth_test, y_hat_holt_winter[:8809])}')

<i><b>Bajó un 5.98% de error respecto al proyecto</b></i>

Predicciones diarias al principio y final del periodo de predicción: 

In [ ]:
# Longitud de intervalo
interval = 24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = exp_smooth_test.iloc[:interval].index, exp_smooth_test.iloc[:interval]
x_pred, y_pred = y_hat_holt_winter.iloc[:interval].index, y_hat_holt_winter.iloc[:interval]

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Holt-Winter Predicción diaria de las primeras {interval} hs de demanda',
                  xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='Energy Demand [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las primeras {interval} horas: {mape(y_true, y_pred)}')

<i><b>Subió un 0.15% de error respecto al proyecto</b></i>

In [ ]:
# Longitud de intervalo
interval = -24*7

# Necesitamos adaptar al intervalo las variables a usarse, 
# dado que la predicción se hizo por intervalos de 24*365
x_true, y_true = exp_smooth_test.iloc[interval:].index, exp_smooth_test.iloc[interval:]
x_pred, y_pred = y_hat_holt_winter[:-8809].iloc[interval:].index, y_hat_holt_winter[:-8809].iloc[interval:]

# Grafica
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_true, y=y_true,
                         mode = 'lines',
                         name = 'Test - Ground Truth'))
fig.add_trace(go.Scatter(x=x_pred, y=y_pred,
                         mode = 'lines',
                         name = 'Test - Prediction'))
# Ajustes varios sobre la grafica
fig.update_traces(line=dict(width=0.9))
fig.update_layout(title=f'Holt-Winter Predicción diaria de las ultimas {abs(interval)} hs de demanda',
                  xaxis_title='Fecha (mm dd, yyyy, hh, MM)',
                  yaxis_title='Demanda [MW]')
fig.show()

# Eficacia 
print(f'MAPE para el intervalo en las ultimas {abs(interval)} horas: {mape(y_true, y_pred)}')

<i><b>Bajó un 3.52% de error respecto al proyecto</b></i>

### Auto-correlograma & Auto-correlograma parcial

In [ ]:
# fig,ax = plt.subplots(2,1,figsize=(20,10))
# fig = sm.graphics.tsa.plot_acf( train['demand_in_MW'].diff().dropna(), lags=72, ax=ax[0])
# fig = sm.graphics.tsa.plot_pacf(train['demand_in_MW'].diff().dropna(), lags=72, ax=ax[1])
# plt.show()